In [1]:
import os, re, sys
import subprocess as sp
import random, string
import numpy as np
from .utils import *
from .pfunc import pfunc

DEBUG=False

# load package locations from yaml file, watch! global dict
package_locs = load_package_locations()

def bpps(sequence, package='vienna', constraint=None, pseudo=False,
         T=37, coaxial=True, linear=False, dna=False,
        motif=None, dangles=True,param_file=None,reweight=None, beam_size=100, DEBUG=False):
    ''' Compute base pairing probability matrix for RNA sequence.
    Args:
    sequence (str): nucleic acid sequence
    T (float): temperature (Celsius)
    linear (bool): call LinearPartition to estimate Z in Vienna or Contrafold
    constraint (str): structure constraint (functional in vienna, contrafold, rnastructure)
    motif (str): argument to vienna motif
    pseudo (bool): (NUPACK only) include pseudoknot calculation
    dangles (bool): dangles or not, specifiable for vienna, nupack
    dna (bool): (NUPACK only) use SantaLucia 1998 parameters for DNA
    coaxial (bool): coaxial stacking or not, specifiable for rnastructure, vfold
    noncanonical(bool): include noncanonical pairs or not (for contrafold, RNAstructure (Cyclefold))
    beam size (int): Beam size for LinearPartition base pair calculation.
    DEBUG (bool): Output command-line calls to packages.
    Possible packages: 'vienna_2', 'vienna_1','contrafold_1','contrafold_2',
    'nupack_95','nupack_99','rnasoft_2007','rnasoft_1999','rnastructure','vfold_0','vfold_1'
 
    Returns
    array: NxN matrix of base pair probabilities
  '''
    try:  # like an if, else statement
        pkg, version = package.lower().split('_')
    except:
        pkg, version = package, None
        
    if motif is not None and pkg != 'vienna':
        raise ValueError('motif option can only be used with Vienna.')

    if pseudo and pkg != 'nupack':
        raise ValueError('pseudoknot option only implemented with Nupack.')

    if not dangles and pkg not in ['vienna','nupack']:
        print('Warning: %s does not support dangles options' % pkg)
    if not coaxial and pkg not in ['rnastructure','vfold']:
        print('Warning: %s does not support coaxial options' % pkg)
    if linear and pkg not in ['vienna','contrafold','eternafold']:
        print('Warning: LinearPartition only implemented for vienna, contrafold, eternafold.')

    if pkg=='nupack':
        return bpps_nupack_(sequence, version = version, dangles = dangles, T = T, pseudo=pseudo, dna=dna)
    elif pkg=='vfold':
        return bpps_vfold_(sequence, version = version, T = T, coaxial = coaxial)
    else:

        _, tmp_file = pfunc(sequence, package=package, bpps=True, linear=linear,
            motif=motif, constraint=constraint, T=T, coaxial=coaxial,
             dangles=dangles, param_file=param_file,reweight=reweight, beam_size=beam_size, DEBUG=DEBUG)
        
        ### to add threshknot:
            # add threshknot=threshknot

        if linear:
            #parse linearpartition output
            return bpps_linearpartition_(sequence, tmp_file)   # if package is specified as linear partition, the bpps_linearpartition function will be called
        else:

            if 'contrafold' in package:
                return bpps_contrafold_(sequence, tmp_file)
            if package=='eternafold':
                return bpps_contrafold_(sequence, tmp_file)                
            elif 'vienna' in package:
                return bpps_vienna_(sequence, tmp_file)
            elif 'rnasoft' in package:
                return bpps_rnasoft_(sequence, tmp_file)
            elif 'rnastructure' in package:
                return bpps_rnastructure_(sequence, tmp_file, coaxial=coaxial)

            else:
                raise RuntimeError('package not yet implemented')

ImportError: attempted relative import with no known parent package

In [2]:
def bpps_linearpartition_(sequence, tmp_file):

    fname = tmp_file

    probs=np.zeros([len(sequence), len(sequence)]) 
    # zero function returns an array of the specified dimensions filled with zeroes

    for line in open(fname,'r').readlines():
        if len(line.strip())>0:     # strip command removes leading and trailing characters; if the line still exists, the function proceeds
            first_ind, second_ind, p = line.strip().split(' ')   # how are there multiple names for this variable separated by spaces?
            first_ind = int(first_ind)-1
            second_ind = int(second_ind)-1
            p = float(p)
            probs[first_ind, second_ind] = p
            probs[second_ind, first_ind] = p

    os.remove(fname)

    return probs    # creates the base pair probability matrices with voodoo math magic